In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 300자 이하로, 5개만 확인

import pandas as pd
import json
import http.client

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/testapp/v1/api-tools/summarization/v2/1dce3880ba404d99b9b0ec378d619870', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['text']
        else:
            return f"Error: {res['status']['message']}"

if __name__ == '__main__':
    completion_executor = CompletionExecutor(
        host='clovastudio.apigw.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiYyJMHWjyWPH1xxOOns//FCEv9a5ykBpcLWBYwWsr3lZ5',
        api_key_primary_val='86MAOcYO4FTl26KRIzE47ozcpFvr7SZiIaLPAt5k',
        request_id='8934ef81-8100-42e2-8fb1-7c93bbb369ef'
    )

    csv_file = '/content/drive/MyDrive/미래에셋/economy_articles.csv'

    try:
        df = pd.read_csv(csv_file, encoding='cp949')
        df = df.head(5)

        for index, row in df.iterrows():
            title = row['Title']
            content = row['Content']

            request_data = {
                "texts": [content],  # 기사의 내용을 직접 사용
                "segMinSize": 300,
                "includeAiFilters": True,
                "autoSentenceSplitter": True,
                "segCount": -1,
                "segMaxSize": 1000
            }

            response_text = completion_executor.execute(request_data)

            # 요약 결과가 300자를 넘을 경우 다시 요약
            while len(response_text) > 300:
                request_data['texts'] = [response_text]  # 이전 요약 결과를 입력으로 다시 요청
                response_text = completion_executor.execute(request_data)

            print(f"제목: {title}")
            print(f"요약 결과: {response_text}")
            print("=" * 50)

    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {csv_file}")
    except Exception as e:
        print(f"오류 발생: {str(e)}")

제목: 인구 감소 위기는 청소년 세대의 문제
요약 결과: - 우리나라 인구 위기와 관련한 뉴스가 연일 쏟아짐
- 대통령이 저출산고령사회위원회 회의를 주재하며 인구 국가비상사태를 선언함
- 인구 감소 속도를 최대한 늦추고 경제 외형이 작아질 수 있는 시대에 적응할 사회시스템을 만들어가는 노력이 필요함
- 인구 감소가 전체 국민소득을 줄일 순 있어도 혁신이 성공하면 1인당 소득이 오히려 증가할 수 있음
- 인구 감소와 지방 소멸은 동전의 양면임
- 함께 풀어야 인구 위기를 넘을 수 있음
제목: 사회보험 지속가능성 '제로'…국민 부담, 어디까지 가능할까
요약 결과: - 정부가 국민연금, 건강보험 등 8대 사회보험과 기초연금, 기초생활보장 등 공공부조를 아우른 사회보장제도의 40년 뒤 재정 전망을 추계해 오는 10월께 공개하기로 함
- 고령화로 복지지출이 급격히 불어나는 가운데 각종 복지제도의 지속가능성을 점검하기 위해서임
- 우리나라의 사회복지제도는 지속 불가능함
- 기초연금과 건강보험 재정수지가 심각함
- 저출산과 고령화의 근본적 원인은 저출산과 고령화에 있음
- 복지제도의 지속가능성을 높이기 위해 어떤 방법이 있을지 고민해봐야 함
제목: 현대차, 60세 정년 후 2년간 '계속 고용'
요약 결과: - 노사가 정년 연장 관련 TF를 꾸려 내년 상반기까지 정년 연장 관련 개선 방안을 논의할 계획임
- 현대자동차가 생산직(기술직·정비직) 근로자에 대해 ‘정년 연장 관련 태스크포스(TF)’를 꾸리기로 한 건 단순히 노동조합의 요구 때문만은 아님
- 저출생 여파로 생산연령인구(15~64세)가 가파르게 줄고 있는 만큼 안정적인 노동 인력 확보는 기업 경쟁력을 위해 필수불가결한 조건임
제목: 원가 제때 반영 않는 전기·가스요금, 괜찮은가
요약 결과: - 공공요금을 올리면 다른 상품이나 서비스 가격도 올라 고물가를 더 자극한다는 현실적 이유도 있음
- 공기업이 누적 적자로 제 기능을 못하면 결국 국민 부담임
- 고물가를 잡겠다고 장담한 정부의 책무를 공기업이 흡수해줘야 함
- 

In [ ]:
# 5개 요약 저장


import pandas as pd
import json
import http.client

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/testapp/v1/api-tools/summarization/v2/1dce3880ba404d99b9b0ec378d619870', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['text']
        else:
            return f"Error: {res['status']['message']}"

if __name__ == '__main__':
    completion_executor = CompletionExecutor(
        host='clovastudio.apigw.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiYyJMHWjyWPH1xxOOns//FCEv9a5ykBpcLWBYwWsr3lZ5',
        api_key_primary_val='86MAOcYO4FTl26KRIzE47ozcpFvr7SZiIaLPAt5k',
        request_id='8934ef81-8100-42e2-8fb1-7c93bbb369ef'
    )

    csv_file = '/content/drive/MyDrive/미래에셋/economy_articles.csv'
    output_file = '/content/drive/MyDrive/미래에셋/summary_results.csv'

    try:
        df = pd.read_csv(csv_file, encoding='cp949')
        df = df.head(5)  # 상위 5개 데이터만 선택

        results = []

        for index, row in df.iterrows():
            title = row['Title']
            content = row['Content']

            request_data = {
                "texts": [content],  # 기사의 내용을 직접 사용
                "segMinSize": 300,
                "includeAiFilters": True,
                "autoSentenceSplitter": True,
                "segCount": -1,
                "segMaxSize": 1000
            }

            response_text = completion_executor.execute(request_data)

            # 요약 결과가 300자를 넘을 경우 다시 요약
            while len(response_text) > 300:
                request_data['texts'] = [response_text]  # 이전 요약 결과를 입력으로 다시 요청
                response_text = completion_executor.execute(request_data)

            results.append({'Title': title, 'Summary': response_text})

        # 결과를 DataFrame으로 변환하여 CSV로 저장
        summary_df = pd.DataFrame(results)
        summary_df.to_csv(output_file, index=False, encoding='cp949')

        print(f"요약 결과가 {output_file}에 저장되었습니다.")

    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {csv_file}")
    except Exception as e:
        print(f"오류 발생: {str(e)}")


요약 결과가 /content/drive/MyDrive/미래에셋/summary_results.csv에 저장되었습니다.


In [ ]:
# 전체 요약, csv 파일 형태로 저장

import pandas as pd
import json
import http.client

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/testapp/v1/api-tools/summarization/v2/1dce3880ba404d99b9b0ec378d619870', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['text']
        else:
            return f"Error: {res['status']['message']}"

if __name__ == '__main__':
    completion_executor = CompletionExecutor(
        host='clovastudio.apigw.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiYyJMHWjyWPH1xxOOns//FCEv9a5ykBpcLWBYwWsr3lZ5',
        api_key_primary_val='86MAOcYO4FTl26KRIzE47ozcpFvr7SZiIaLPAt5k',
        request_id='8934ef81-8100-42e2-8fb1-7c93bbb369ef'
    )

    csv_file = '/content/drive/MyDrive/미래에셋/economy_articles.csv'
    output_file = '/content/drive/MyDrive/미래에셋/summary_results.csv'

    try:
        df = pd.read_csv(csv_file, encoding='cp949')
        results = []

        for index, row in df.iterrows():
            title = row['Title']
            content = row['Content']

            request_data = {
                "texts": [content],  # 기사의 내용을 직접 사용
                "segMinSize": 300,
                "includeAiFilters": True,
                "autoSentenceSplitter": True,
                "segCount": -1,
                "segMaxSize": 1000
            }

            response_text = completion_executor.execute(request_data)

            # 요약 결과가 300자를 넘을 경우 다시 요약
            while len(response_text) > 300:
                request_data['texts'] = [response_text]  # 이전 요약 결과를 입력으로 다시 요청
                response_text = completion_executor.execute(request_data)

            results.append({'Title': title, 'Summary': response_text})

        # 결과를 DataFrame으로 변환하여 CSV로 저장
        summary_df = pd.DataFrame(results)
        summary_df.to_csv(output_file, index=False, encoding='cp949')

        print(f"요약 결과가 {output_file}에 저장되었습니다.")

    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {csv_file}")
    except Exception as e:
        print(f"오류 발생: {str(e)}")

